In [7]:
import socket
socket.gethostname()

'calebs-skylake-ubuntu-1604'

In [8]:
import os
import sys
import numpy as np
import nept
import nelpy as nel
import matplotlib.pyplot as plt
%matplotlib inline

/home/kemerelab/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.


In [9]:
dataroot = '/home/kemerelab/Data/motivational-t'

# Find all rats and days

In [10]:
from glob import glob
alldirs = glob(os.path.join(dataroot,'*/'))


In [11]:
import re 
datadirs = [re.match(r'.*(R\d\d\d-\d\d\d\d-\d\d-\d\d)', x).group(1) for x in alldirs 
     if re.match(r'.*(R\d\d\d-\d\d\d\d-\d\d-\d\d)', x)]

In [4]:
datadirs

NameError: name 'datadirs' is not defined

In [12]:
# fix annoying 'display last result only' behavior
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [13]:
# Info from ExpKeys
CONST_conv_fact = {}
CONST_conv_fact['R042'] = [2.9176, 2.3794]
CONST_conv_fact['R050'] = [2.6853, 2.6169]
CONST_conv_fact['R064'] = [2.6497, 2.6485]


In [29]:
import zipfile
import scipy.io as sio


Data = {}
for ddir in datadirs:
    dayinf = re.match(r'(R\d\d\d)-(\d\d\d\d-\d\d-\d\d)', ddir)
    anim = dayinf.group(1)
    day = dayinf.group(2)
    print(anim,day)
    if anim not in Data:
        Data[anim] = {}
    
    # define some testing data file locations
    fn = os.path.join(dataroot,ddir) # need function that makes list of all folders
    
    
    fn_vt = ddir + '-VT1.zip'
    fn_vt = os.path.join(dataroot, ddir, fn_vt)
    print(fn_vt)
    with zipfile.ZipFile(fn_vt,"r") as zip_ref:
        zip_ref.extractall("/tmp")
    tmp_vt = '/tmp/VT1.nvt'
    
    # Load position (.nvt) from this experiment
    position = nept.load_position(tmp_vt,CONST_conv_fact[anim])
    
    # vt is a 2-D AnalogSignalArray -- note has bigger gaps in data b/c tracking LEDs get obscured
    vt_nelpy = np.stack((position.x,position.y),1)
    vt_nelpy.shape
    # with warnings.catch_warnings():
    #    warnings.simplefilter("ignore")
    vt_nelpy = nel.AnalogSignalArray(ydata=vt_nelpy.T,timestamps=position.time,merge_sample_gap=10,fs=30)

    Data[anim][day] = {}
    Data[anim][day]['pos'] = vt_nelpy
    del position
    
    spikes = nept.load_spikes(os.path.join(dataroot, ddirfd))
    spikes_list = [ss.time for ss in spikes]

    
    # load ripple "candidates"
    fn_cand = ddir + '-candidates.mat'
    cand = sio.loadmat(os.path.join(dataroot,ddir,fn_cand))
    
    break


R042 2013-08-16
/home/kemerelab/Data/motivational-t/R042-2013-08-16/R042-2013-08-16-VT1.zip


(120390, 2)

/home/kemerelab/Code/nelpy/nelpy/core/_analogsignalarray.py:360: UserWarning: creating support from time and sampling rate, fs!
/home/kemerelab/Code/nelpy/nelpy/utils.py:549: UserWarning: some steps in the data are smaller than the requested step size.


In [ ]:
Data = {}
Data['R042'] = {}
Data['R042']['2013-08-18'] = {}

In [ ]:
# Load position (.nvt) from this experiment
position = nept.load_position(os.path.join(dataroot, fd, fn_vt),CONST_conv_fact)

# Plot the position
plt.plot(position.x, position.y, 'k.', ms=1)
plt.show()

In [ ]:
# load spikes
spikes = nept.load_spikes(os.path.join(dataroot, fd))
spikes_list = [ss.time for ss in spikes]

# Plot the spikes
# for idx, spiketrain in enumerate(spikes):
#     plt.plot(spiketrain.time, np.ones(len(spiketrain.time))+idx, '|');
# plt.show()

In [ ]:
# load candidates
fn_cand = 'R042-2013-08-18-candidates.mat'
import scipy.io as sio
cand = sio.loadmat(os.path.join(dataroot,fd,fn_cand))

In [ ]:
tstart = cand['evt'][0][0]['tstart']
tend = cand['evt'][0][0]['tend']

Data['R042']['2013-08-18']['event-candidates'] = nel.EpochArray(np.hstack([tstart, tend]))

In [ ]:
metadata = sio.loadmat(os.path.join(dataroot,fd,'R042-2013-08-18-metadata.mat'))

In [ ]:
taskvars = metadata['metadata'][0][0]['taskvars'][0][0]
metavars = list(taskvars.dtype.fields.keys())
iv_metavars = list(filter(lambda mv: 'iv' in mv, metavars))

for mv in iv_metavars:
    tstart = taskvars[mv][0][0]['tstart']
    tend = taskvars[mv][0][0]['tend']
    Data['R042']['2013-08-18'][mv] = nel.EpochArray(np.hstack([tstart, tend]))
    
sequence = [choice[0] for choice in taskvars['sequence'][0]]
Data['R042']['2013-08-18']['sequence'] = sequence

In [ ]:
taskvars.dtype.fields.keys()

In [ ]:
Data['R042']['2013-08-18'].keys()

In [ ]:
nel_spikes = nel.SpikeTrainArray(spikes_list)

Data['R042']['2013-08-18']['spikes'] = nel_spikes

# Figure out bounds of arms

In [ ]:
runpos = vt_nelpy[Data['R042']['2013-08-18']['trial_iv'],:]

fig, ax = plt.subplots(1,1, figsize = (8, 8))

xvals, yvals = runpos.simplify(n_points=5000).asarray().yvals
ax.set_aspect('equal')
ax.plot(xvals, yvals, lw=0.25, color='k', alpha=0.5)

from shapely.geometry import box
from descartes.patch import PolygonPatch # conda install -c conda-forge descartes

RightArmCoords = [150,0,225,85]
b_right = box(*RightArmCoords)
ax.add_patch(PolygonPatch(b_right))

LeftArmCoords = [150,120,225,220]
b_left = box(*LeftArmCoords)
ax.add_patch(PolygonPatch(b_left))

Data['R042']['2013-08-18']['LeftArmRect'] = LeftArmCoords
Data['R042']['2013-08-18']['RightArmRect'] = RightArmCoords

In [ ]:
import nelpy.io
MVDMData = nel.ResultsContainer(Data=Data,
                                    description='Test dataset - R042 2013-08-18.\n')

In [ ]:
MVDMData.save_pkl('/home/kemerelab/Data/MVDMData.nel')